## Predictive Model for Water Level of the Chattooga River

TODO: ADD Image of the model here

As a whitewater kayaker, I take many trips to the Chattooga River. The water level is determined by the amount of rainfall in the area. It is important to know the water level of the river before making the trip, because if the water level is too high or too low, the river is not safe to take trips out. The goal of this project is to create a predictive model for the water level of the Chattooga River. This model will be used to predict the water level of the river for the next 24 hours. This will allow for an exploration of how predictive models work, using research to solve a problem, and provide a useful tool for me.

## Outline

## Data Collection

Data collection consited of two parts, the water level and the percipitation data. 

#### Water Level

The first part was collecting data from the USGS site. The USGS website contained data only back to 2007, which is a limiting factor, but it is the best data available for the Chattooga River. The data would specfically be take from one site, Chattooga River Near Clayton, GA, since this is the one most often used by us kayakers. Data is in the form of a text file, listing data about water levels every 15 min.

#### Weather Data

The second part of the data is the weather data. Rabun county was used as it is the location of the gauge we are using for the Chattooga River. The data was in the form of a csv, and contained data from 2007 to 2023, and contains data about weather pulled every day. 

In [1]:
print('Model')

Model
